In [1]:
import os
from datasets import Dataset
import h5py
import re
from transformers import T5Tokenizer

file_name = ""  # "_dummy"

In [ ]:
with h5py.File(f"../tmp/data/pssm/pssm_data{file_name}.h5", "r") as f:
    print(len(list(f.keys())))
    print(list(f["1k87A01"].attrs))
    print()
    print(f["1k87A01"].attrs["name"])
    print(f["1k87A01"].attrs["sequence"])
    print(list(f["1k87A01"].keys()))
    print(type(f["1k87A01"]["320_0"][:]))
    print(f["1k87A01"]["320_0"][:].shape)
    print(f["1k87A01"]["320_0"][:])
    print()
    # for x in range(1000):
    # for x in list(f["1k87A01"].keys()):
    #     # asdf = f["1k87A01"][x][:]
    #     print(f["1k87A01"][x][:].shape)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path="Rostlab/prot_t5_xl_uniref50",
    do_lower_case=False,
    use_fast=True,
    legacy=False,
)


def gen():
    with h5py.File(f"../tmp/data/pssm/pssm_data{file_name}.h5", "r") as f:
        for protein_id in f.keys():
            for trajectory_pssm in f[protein_id].keys():
                sequence_processed = " ".join(list(re.sub(r"[UZOB]", "X", f[protein_id].attrs["sequence"])))
                sequence_tokenized = tokenizer(text=sequence_processed, padding=False, truncation=False)
                input_ids = sequence_tokenized["input_ids"]
                attention_mask = sequence_tokenized["attention_mask"]
                yield {
                    "name": f[protein_id].attrs["name"] + "_" + trajectory_pssm,
                    "sequence": f[protein_id].attrs["sequence"],
                    "sequence_processed": sequence_processed,
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "pssm_features": f[protein_id][trajectory_pssm][:],
                }


ds = Dataset.from_generator(gen)

ds.save_to_disk(f"../tmp/data/pssm/pssm_dataset{file_name}")

In [ ]:
from datasets import load_from_disk

ds = load_from_disk(f"../tmp/data/pssm/pssm_dataset{file_name}")

print("Dataset created with", len(ds), "examples")
print("\nDataset:", ds)
print()
print(ds[0])
print(type(ds[0]["name"]))
print(type(ds[0]["sequence"]))
print(type(ds[0]["sequence_processed"]))
print(type(ds[0]["pssm_features"]))

In [ ]:
ds = ds.filter(lambda x: x["name"].endswith("_0"))
print(ds)
ds.save_to_disk(f"../tmp/data/pssm/pssm_dataset{file_name}_0_only")

In [ ]:
ds = ds.filter(lambda x: len(x["sequence"]) < 512)
print(ds)
ds.save_to_disk(f"../tmp/data/pssm/pssm_dataset{file_name}_0_only_shorter_512")
